In [3]:
from data_sampler import WeightedSampler
from dataset import CARLADataset, CARLADatasetMultiProcessing
from utils import create_metadata_df, train_test_split
from data_preprocessing import preprocessing
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import torch
import os
import pandas as pd
import matplotlib.pyplot as plt
from zipfile import ZipFile, ZIP_DEFLATED

from tqdm import tqdm
import cv2

/Users/julianvonklitzing/miniforge3/envs/carla/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# path_data = os.path.join("..", "data", "Dataset Ege")
path_data = os.path.join("..", "data", "data")
config = {"used_inputs": ["rgb","measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake", "command"],
        "seq_len": 1
        }

df_meta_data = create_metadata_df(path_data, config["used_inputs"])
df_meta_data_train, df_meta_data_test = train_test_split(df_meta_data, towns={"train": ["Town04", "Town06"], "test": ["Town05"]})
dataset_train = CARLADataset(root_dir=path_data, df_meta_data=df_meta_data_train, config=config)
dataset_test = CARLADataset(root_dir=path_data, df_meta_data=df_meta_data_test, config=config)
# dataset_mp = CARLADatasetMultiProcessing(root_dir=path_data, config=config)
# weighted_sampler = WeightedSampler(dataset=dataset)
dl_train = DataLoader(dataset=dataset_train, batch_size=64, num_workers=0)
dl_test = DataLoader(dataset=dataset_test, batch_size=64, num_workers=0)

In [5]:
len(df_meta_data_train) / len(df_meta_data)

0.3741433792000494

In [6]:
dataset_train.get_statistics()

,rgb_in_GB,measurements_in_GB,time_hours
0,20.141904,0.425886,13.451806


In [10]:
dataset_test.get_statistics()

,rgb_in_GB,measurements_in_GB,time_hours
0,11.734764,0.24525,7.74125


In [7]:
count = 0
for batch in tqdm(dl_train):
    count += 1
print(count)

100%|██████████| 1514/1514 [16:43<00:00,  1.51it/s]

1514


In [ ]:
count = 0
for batch in tqdm(dl_test):
    count += 1
print(count)

In [46]:
# Save images to video to discover/understand the scenes 
img_folder = "../data/data/rr_dataset_23_11/Routes_clipped_Town04_rr_Seed0/clipped_Town04_rr_route0_11_23_15_49_32/rgb"
images_path = sorted(os.listdir(img_folder))
video = cv2.VideoWriter("first_video.avi", 0, 2, (960, 160))
for image in images_path:
    video.write(cv2.imread(os.path.join(img_folder, image)))